In [10]:
import pandas as pd
import os 
import cv2
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
# identify IDs of controls and Alzheimer patients
demographics = pd.read_excel('oasis_cross-sectional.xls', sheetname=3)
df = demographics.dropna(how='any')
df_columns = list(demographics.columns)
X_columns = [column_name for column_name in df_columns if column_name != 'CDR']
X_columns = [column_name for column_name in X_columns if column_name != 'MMSE']
Xdf = df.reindex(columns=X_columns)
X = Xdf.values

In [12]:
# Define function to get list of pngs
pngs_path='OASIS_MR1_pngs'
def getaxialPNG(path):
    l = []
    axialslice90_files = []
    axialslice91_files = []
    axialslice92_files = []
    axialslice93_files = []
    axialslice94_files = []
    axialslice95_files = []
    axialslice96_files = []
    axialslice97_files = []
    axialslice98_files = []
    axialslice99_files = []
    for root, directories, filenames in os.walk(path):
        
        for filename in filenames:
            if ".90." in filename: 
                axialslice90_files.append(os.path.join(root, filename))
            if ".91." in filename: 
                axialslice91_files.append(os.path.join(root, filename))
            if ".92." in filename: 
                axialslice92_files.append(os.path.join(root, filename))
            if ".93." in filename: 
                axialslice93_files.append(os.path.join(root, filename))
            if ".94." in filename: 
                axialslice94_files.append(os.path.join(root, filename))
            if ".95." in filename: 
                axialslice95_files.append(os.path.join(root, filename))
            if ".96." in filename: 
                axialslice96_files.append(os.path.join(root, filename))
            if ".97." in filename: 
                axialslice97_files.append(os.path.join(root, filename))
            if ".98." in filename: 
                axialslice98_files.append(os.path.join(root, filename))
            if ".99." in filename: 
                axialslice99_files.append(os.path.join(root, filename))

    l = list(zip(axialslice90_files, axialslice91_files, axialslice92_files, axialslice93_files, axialslice94_files, axialslice95_files, axialslice96_files, axialslice97_files, axialslice98_files, axialslice99_files))

    return ((np.asarray(l)))

axial_files0 = getaxialPNG(pngs_path)
axial_X_files = np.take(axial_files0, indices=df.index.values, axis=0)

In [13]:
X_id, X_sex, X_handedness, X_age, X_education, X_SES, X_eTIV, X_nWBV, X_ASF = zip(*X)

def sex_translator(X_sex):
    X_sex_binary = []
    for x in X_sex:
        if x == 'M':
            X_sex_binary.append(1)
        else:
            X_sex_binary.append(-1)
    
    return(np.asarray(X_sex_binary))
           
def hand_translator(X_handedness):
    X_hand_binary = []
    for x in X_handedness:
        if x == 'R':
            X_hand_binary.append(1)
        else:
            X_hand_binary.append(-1)
    
    return(np.asarray(X_hand_binary))      

X_sex_binary = sex_translator(X_sex)
X_hand_binary = hand_translator(X_handedness)

In [14]:
Y_CDR_columns = [column_name for column_name in df_columns if column_name == 'CDR']
Y_CDR_df = df.reindex(columns=Y_CDR_columns)
Y_CDR = Y_CDR_df.values

Y_MMSE_columns = [column_name for column_name in df_columns if column_name == 'MMSE']
Y_MMSE_df = df.reindex(columns=Y_MMSE_columns)
Y_MMSE = Y_MMSE_df.values

CDR_threshold_0 = 0
CDR_threshold_0point5 = 0.5
CDR_threshold_1 = 1

MMSE_threshold_24 = 24
MMSE_threshold_18 = 18

def CDR_probable_AD_thresholder(Y_CDR, threshold_value):
    Y_CDR_binary = []
    for y in Y_CDR:
        if y > threshold_value:
            Y_CDR_binary.append(1)
        else:
            Y_CDR_binary.append(-1)
        
    return(np.asarray(Y_CDR_binary))

def MMSE_probable_Dementia_thresholder(Y_MMSE, threshold_value):
    Y_MMSE_binary = []
    for y in Y_MMSE:
        if y < threshold_value:
            Y_MMSE_binary.append(1)
        else:
            Y_MMSE_binary.append(-1)
        
    return(np.asarray(Y_MMSE_binary))

Y_CDR_binary = CDR_probable_AD_thresholder(Y_CDR, CDR_threshold_0)
Y_MMSE_binary = MMSE_probable_Dementia_thresholder(Y_MMSE, MMSE_threshold_24)

In [6]:
# Prepare images
def prepPNG(imgs):
    # initialize list
    l = []
    
    # load image
    for i in imgs:        
        tmp = cv2.IMREAD_GRAYSCALE(" ".join(i)) # load
        tmp = cv2.resize(np.array(tmp), (224,224))
        l.append(tmp)
    
    return l

images = np.array(prepPNG(imgs))

NameError: name 'imgs' is not defined